### 누락데이터 치환

- 데이터셋의 품질을 높일 목적으로 누락데이터를 무작정 삭제해 버린다면 어렵게 수집한 데이터를 활용하지 못하게 된다.
- 데이터 분석의 정확도는 데이터의품질 외에도 제공된는 데이터의 양에 의해 상당한 영향을 받는다. 따라서 데이터 중에서 일부가 누락되어 있더라도 나머지 데이터를 최대한 살려서 데이터 분석에 활용하는 것이 좋은 결과를 얻는 경우가 많다.
- 누락 데이터를 바꿔서 대체할 값으로는 데이터 분포와 특성을 잘 나타낼 수 있는 평균값, 최빈값 들이 있다. 
- 판다스에서 fillna() 메소드로 편리하게 처리를 한다. 이 메소드는 새로운 객체를 반환하기 때문에 원본 객체를 변경하려면 inplace=True옵션을 추가해야함.
- 평균값 혹은 중간값을 변수로 담아서 fillna() 메소드에 인자로 전달하면 NaN값이 치환이 된다.


In [1]:
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')

In [3]:
print(df['age'].head(10))

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: age, dtype: float64


In [6]:
#평균값으로 값 대치하기
mean_age = df['age'].mean(axis=0)
df['age'].fillna(mean_age, inplace=True)

print(df['age'].head(10))

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64


In [7]:
# 중간값으로 NaN값 대치하기
median_age = df['age'].median(axis=0)
df['age'].fillna(median_age, inplace=True)

print(df['age'].head(10))

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64


수치형 변수는 이런 값으로 대체할 수 있지만 범주형 변수에 대한 값을 최빈값으로 대치하고 싶다면??

In [8]:
print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


In [10]:
# embark_town 열의 NaN 값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = df['embark_town'].value_counts(dropna=True).idxmax() #NaN값을 제외하고 유요한 데이터 개수에서 최대 값을 추출
print(most_freq)
print('\n')

df['embark_town'].fillna(most_freq, inplace=True)
print(df['embark_town'][825:830])

Southampton


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object


NaN 값이 최빈값 Southampton로 치환이 되었다.

실질적으로 데이터셋 중에는 누락 데이터가 NaN값으로 입력되지 않은 경우가 있다(숫자 0,'-','?')이것들과 같이 판다스에서 누락 데이터를 다루려면 replace() 메소드를 활용하여 Numpy에서 지원하는 np.nan 변경 해주는 것이 좋다.    
df.replace('?', np.nan, inplace=True)

데이터셋의 특성상 서로 이웃하고 있는 데이터끼리 유사성을 가질 가능성이 높다. 이럴 때는 앞이나 뒤에서 이웃라고 있는 값으로 치환해 주는 것이 좋다. fillna() 메소드에 method='ffill' ,method='bfill' 옵션을 적용하면 앞,뒤 값으로 치환한다.

In [12]:
# titanic 데이터셋 가져오기
df = sns.load_dataset('titanic')

# embark_town 열의 829행의 NaN 데이터 출력
print(df['embark_town'][825:830])
print('\n')

# embark_town 열의 NaN값을 바로 앞에 있는 828행의 값으로 변경하기
df['embark_town'].fillna(method='ffill', inplace=True)
print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object


825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object


## 중복 데이터 처리

- 데이터프레임에서 각 행은 분석 대상이 갖고 있는 모든 속성에 대한 관측값을 뜻한다. 하나의 데이터셋에서 동일한 관측값이 2개 이상 중복되는 경우 중복 데이터를 찾아서 삭제해야한다.
- 동일한 대상이 중복으로 존재하므로 분석 결과를 왜곡하기 떄문이다.

### 중복 데이터 확인

- duplicated() 메소를 이용한다. 전에 나온 행들과 비교하여 중복되는 행이면 True를 반환하고 처음 나오는 행에 대해서는 False를 반환한다.
- 각 행의 중복여부를 나타내는 불린 시리즈를 반환한다는 것이다.

In [13]:
import pandas as pd

# 중복 데이터를 갖는 데이터프레임 만들기
df = pd.DataFrame({'c1':['a', 'a', 'b', 'a', 'b'],
                  'c2':[1, 1, 1, 2, 2],
                  'c3':[1, 1, 2, 2, 2]})
print(df)
print('\n')

# 데이터프레임 전체 행 데이터 중에서 중복값 찾기
df_dup = df.duplicated()
print(df_dup)
print('\n')

# 데이터프레임의 특정 열 데이터에서 중복값 찾기
col_dup = df['c2'].duplicated()
print(col_dup)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


0    False
1     True
2    False
3    False
4    False
dtype: bool


0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool


### 중복 데이터 제거

- 중복 데이터를 제거하는 명령에는 drop_duplicates() 메소드를 사용한다.
- 중복되는 행을 제거하고 고유한 관측값을 가진 행들만 남긴다.
- 원본 객체를 변경하려면 inplace=True 옵션을 추가한다.

In [14]:
# 중복 데이터를 갖는 데이터프레임 만들기
df = pd.DataFrame({'c1':['a', 'a', 'b', 'a', 'b'],
                  'c2':[1, 1, 1, 2, 2],
                  'c3':[1, 1, 2, 2, 2]})
print(df)
print('\n')

# 데이터프레임에서 중복 행을 제거
df2 = df.drop_duplicates()
print(df2)
print('\n')

# c2, c3열을 기준으로 중복 행을 제거
df3 = df.drop_duplicates(subset=['c2', 'c3'])
print(df3)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2


drop_duplicates() 메소드의 subset 옵션에 '열이름의 리스트'를 전달할 수 있다. 데이터의 중복 여부를 판별할 때 , subset옵션에 해당하는 열을 기준으로 판단하여 중복을 제거한다.

## 데이터 표준화

- 여러 곳에서 수집한 자료들은 단위 선택, 대소문자 구분, 약칭 활용등 여러가지 원인에 의해 다양한 형태로 표현된다.
- 서로 다른 단위가 섞여 있거나 같은 대상을 다른 형식으로 표현한 경우가 의외로 많다.
- 이처럼 동일한 대상을 표현하는 방법에는 차이가 있으면 분석의 정확도는 현저히 낮아진다. 따라서 데이터 포멧을 일관성 있게 표준화하는 작업이 필요하다.

### 단위 환산

같은 데이터 셋 안에서 서로 다른 측정 단위를 사용한다면, 전체 데이터의 일관성 측면에서 문제가 발생한다.   
따라서 측정 단위를 동일하게 맞출 필요가 있다. 예제에서는 미국에서 쓰는 갤런당 마일단위로 연비를 표시하는 것을 리터당 킬로미터로 단위를 변환해보자.

In [15]:
df = pd.read_csv('./auto-mpg.csv')
print(df.head())
print('\n')

# mpg(mile per gallon)를 kpl(kilometer per liter)로 변환 (mpg_to_kpl = 0.425)
mpg_to_kpl = 1.60934 / 3.78541

# mpg 열에 0.425를 곱한 결과를 새로운 열(kpl)에 추가
df['kpl'] = df['mpg'] * mpg_to_kpl
print(df.head(3))    
print('\n')

# kpl 열을 소수점 아래 둘째 자리에서 반올림 
df['kpl'] = df['kpl'].round(2)
print(df.head(3))     

    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  15.0          8         350.0        165    3693          11.5          70   
2  18.0          8         318.0        150    3436          11.0          70   
3  16.0          8         304.0        150    3433          12.0          70   
4  17.0          8         302.0        140    3449          10.5          70   

   origin                   car name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  


    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  15.0          8         350.0        165    3693          11.5          70   
2  18.0          8         318.0  

### 자료형 변환

- 숫자가 문자열로 저장된 경우 숫자형으로 변환을 해줘야한다.
- 먼저 dtypes 속성을 사용하여 데이터프레임을 구성하는 각 열의 자료형을 확인한다.(혹은 info() 메소드로 각 열의 자료형을 확인)

In [16]:
df = pd.read_csv('./auto-mpg.csv')
# 각 열의 자료형 확인
print(df.dtypes)
print('\n')

# 문자열을 뜻하는 object 자료형인 'horsepower' 열의 고유값 확인
print(df['horsepower'].unique())

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight            int64
acceleration    float64
model year        int64
origin            int64
car name         object
dtype: object


['130' '165' '150' '140' '198' '220' '215' '225' '190' '170' '160' '95'
 '97' '85' '88' '46' '87' '90' '113' '200' '210' '193' '?' '100' '105'
 '175' '153' '180' '110' '72' '86' '70' '76' '65' '69' '60' '80' '54'
 '208' '155' '112' '92' '145' '137' '158' '167' '94' '107' '230' '49' '75'
 '91' '122' '67' '83' '78' '52' '61' '93' '148' '129' '96' '71' '98' '115'
 '53' '81' '79' '120' '152' '102' '108' '68' '58' '149' '89' '63' '48'
 '66' '139' '103' '125' '133' '138' '135' '142' '77' '62' '132' '84' '64'
 '74' '116' '82']


엔진 출력의 크기를 나타내는 데이터 만큼 숫자형으로 변환해주는 것이 적절하다. 하지만 문자열로 저장된 이유는 고유값에서 ?가 있기 때문이다.    
?가 포함된 데이터를 NaN 값으로 변환을 하고 dropna() 메소드로 NaN값을 포함하는 모든 행을 삭제를 할 것이다. 그리고 데이터타입을 변환하는 astype() 메소드를 활용하여 실수형 데이터 타입으로 변환하자.

In [21]:
import numpy as np
df['horsepower'].replace('?', np.nan, inplace=True) #?을 NaN 값으로 대치
df.dropna(subset=['horsepower'], axis=0, inplace=True) #horsepower 열 nan값 포함되는 값 행 제거
df['horsepower'] = df['horsepower'].astype('float') # 형변환

print(df['horsepower'].dtypes)
print(df['horsepower'].unique())

float64
[130. 165. 150. 140. 198. 220. 215. 225. 190. 170. 160.  95.  97.  85.
  88.  46.  87.  90. 113. 200. 210. 193. 100. 105. 175. 153. 180. 110.
  72.  86.  70.  76.  65.  69.  60.  80.  54. 208. 155. 112.  92. 145.
 137. 158. 167.  94. 107. 230.  49.  75.  91. 122.  67.  83.  78.  52.
  61.  93. 148. 129.  96.  71.  98. 115.  53.  81.  79. 120. 152. 102.
 108.  68.  58. 149.  89.  63.  48.  66. 139. 103. 125. 133. 138. 135.
 142.  77.  62. 132.  84.  64.  74. 116.  82.]


또한 'origin' 열에서 정수형 데이터 1,2,3이 들어가 있지만 실제로는 국가 이름인'usa, eu, jpn' 을 뜻하기 때문에 각 숫자 데이터를 국가 이름으로 바꿔 보자

In [25]:
# origin 열의 고유값 확인
print(df['origin'].unique())

# 정수형 데이터를 문자형 데이터로 변환 
df['origin'].replace({1:'USA', 2:'EU', 3:'JAPAN'}, inplace=True)

# origin 열의 고유값과 자료형 확인
print(df['origin'].unique())
print(df['origin'].dtypes) 
print('\n')

['USA' 'JAPAN' 'EU']
['USA' 'JAPAN' 'EU']
object




3개 국가 이름이 반복된다. 이처럼 유한 개의 고유값이 반복적으로 나타나는 경우에는 범주형(category) 데이터로 표현하는 것이 효율적이다. 

In [27]:
# origin 열의 문자열 자료형을 범주형으로 변환
df['origin'] = df['origin'].astype('category')     
print(df['origin'].dtypes) 

# 범주형을 문자열로 다시 변환
df['origin'] = df['origin'].astype('str')     
print(df['origin'].dtypes)

category
object


마지막으로 'model_year' 열의 자료형을 살펴보면 모델 출시연도가 숫자로 기록이 되어 있고 , 자료형은 정수형을 나타낸다. 연도를 뜻하기 때문에 숫자형으로 남겨둬도 무방하지만 연도는 시간적인 순서의 의미는 있으나 숫자의 상대적인 크기는 별 의미가 없다. 따라서 데이터는 숫자형태를 갖더라도 자료형은 범주형으로 표현하는 것이 적절하다.

In [28]:

# model year 열의 정수형을 범주형으로 변환
print(df['model year'].sample(3))
df['model year'] = df['model year'].astype('category') 
print(df['model year'].sample(3)) 

63     72
381    82
378    82
Name: model year, dtype: int64
125    74
129    74
120    73
Name: model year, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]


## 범주형(카테고리) 데이터 처리

### 구간 분할

- 데이터 분석 알고리즘에 따라서는 연속 데이터를 그래도 사용하기 보다는 일정한 구간으로 나눠서 분석하는 것이 효율적인 경우가 있다.
- 가격,비용, 효율 등 연속적인 값을 일정한 수준이나 정도로 나타내는 이산적(범주형)으로 값으로 나타내어 구간별 차이를 드러내는 것이다. 
- 연속 변수를 일정한 구간으로 나누고, 각 구간을 범주형 이산 변수로 변환하는 과정을 구간 분할이라고 한다.
- 판다스에서 cut() 메소드를 이용하면 연속데이터를 여러구간으로 나누고 범주형 데이터로 변환할 수 있다.

auto-mpg 데이터셋에서 'horsepower' 열을 구간으로 나누어 '저출력', '보통출력', '고출력' 등 구간으로 나누어서 표시해보자. 3개 구간을 구분하려면 4개의 경계값이 필요하다. 이는 numpy 라이브러리에서 histogram() 함수를 활용한 방법으로 하자.

histogram() 함수는 나누려는 구간(bin)개수를 bins옵션에 입력하면 각 구간에 속하는 값에 속하는 값의 개수(count), 경계값 리스트(bin_dividers)를 반환한다.

In [29]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('./auto-mpg.csv')

# horsepower 열의 누락 데이터('?') 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace=True)      # '?'을 np.nan으로 변경
df.dropna(subset=['horsepower'], axis=0, inplace=True)   # 누락데이터 행을 삭제
df['horsepower'] = df['horsepower'].astype('float')      # 문자열을 실수형으로 변환

# np.histogram 함수로 3개의 bin으로 나누는 경계 값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
print(bin_dividers) 

# 3개의 bin에 이름 지정
bin_names = ['저출력', '보통출력', '고출력']

# pd.cut 함수로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x=df['horsepower'],     # 데이터 배열
                      bins=bin_dividers,      # 경계 값 리스트
                      labels=bin_names,       # bin 이름
                      include_lowest=True)    # 첫 경계값 포함 

# horsepower 열, hp_bin 열의 첫 15행을 출력
print(df[['horsepower', 'hp_bin']].head(15))

[ 46.         107.33333333 168.66666667 230.        ]
    horsepower hp_bin
0        130.0   보통출력
1        165.0   보통출력
2        150.0   보통출력
3        150.0   보통출력
4        140.0   보통출력
5        198.0    고출력
6        220.0    고출력
7        215.0    고출력
8        225.0    고출력
9        190.0    고출력
10       170.0    고출력
11       160.0   보통출력
12       150.0   보통출력
13       225.0    고출력
14        95.0    저출력


### 더미 변수

- 앞에서 'horsepower' 열을 범주형 데이터로 변환을 했는대, 이처럼 카테고리를 나타내는 범주형 데이터를 회귀분석등 머신러닝 알고리즘에 반로 사용할 수 없다. 왜냐면 범주형 데이터는 컴퓨터가 인식 가능한 값이 아니기 떄문이다.
- 이때 필요한 변환 작업이 0또는 1로 표현되는 더비 변수를 사용하는 것이다. 여기서 0,1는 수치의 크기에 의미 보다는 어떤 특성이 있는지 없는지 여부이다.
- 이를 다른 말로 one-hot-encoding이라고도 부른다.
- get_dummies() 함수를 사용한다. 

In [30]:
# hp_bin 열의 범주형 데이터를 더미 변수로 변환
horsepower_dummies = pd.get_dummies(df['hp_bin'])
print(horsepower_dummies.head(15))

    저출력  보통출력  고출력
0     0     1    0
1     0     1    0
2     0     1    0
3     0     1    0
4     0     1    0
5     0     0    1
6     0     0    1
7     0     0    1
8     0     0    1
9     0     0    1
10    0     0    1
11    0     1    0
12    0     1    0
13    0     0    1
14    1     0    0


또한 sklearn 라이브러리를 이용해서 원핫인코딩을 편하게 처리가 가능하다.

In [31]:
# sklern 라이브러리 불러오기
from sklearn import preprocessing    

# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder()       # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()   # one hot encoder 생성

# label encoder로 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))  
print(onehot_labeled)
print(type(onehot_labeled))

# 2차원 행렬로 형태 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1) 
print(onehot_reshaped)
print(type(onehot_reshaped))

# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>
